###Скачивание файлов близкородственной бактерии:

In [ ]:
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.


To activate EDirect for this terminal session, please execute the following:

export PATH=${PATH}:${HOME}/edirect



In [ ]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

In [ ]:
!wget https://raw.githubusercontent.com/DaryaSushkova/hse22_hw2/main/data/gms2.lst

--2022-10-19 18:49:20--  https://raw.githubusercontent.com/DaryaSushkova/hse22_hw2/main/data/gms2.lst
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 221257 (216K) [text/plain]
Saving to: ‘gms2.lst’

gms2.lst            100%[===================>] 216.07K  --.-KB/s    in 0.03s   

2022-10-19 18:49:20 (8.11 MB/s) - ‘gms2.lst’ saved [221257/221257]



###Установка программ:

In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [ ]:
!apt-get install ncbi-blast+

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ncbi-blast+ is already the newest version (2.6.0-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


###Предсказание рибосомальной РНК:

In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
import re

In [ ]:
# Разделение на стренды, так как вырезка будет происходить разным образом.
plus = list()
minus = list()

# Проходимся по записям и находим РНК.
for record in SeqIO.read("T_oleivorans_MIL_1.gbk", "genbank").features:
  if record.type == "rRNA":
    # Строка имеет формат [start:end](strand)
    loc_str = str(record.location)
    print(loc_str)
    # Выбираем часть строки с координатами, отбрасывая стрэнд.
    loc_str = loc_str[:-3:]
    # Преобразуем в набор координат.
    locations = re.compile('[^\d:]').sub('', loc_str).replace(':', ' ').split()
    
    start = int(locations[0])
    end = int(locations[1])

    if (str(record.location).find('-') == -1):
      plus.append([start, end])
    else:
      minus.append([int(locations[0]), int(locations[1])])

print(plus)
print(minus)

[341494:343033](+)
[343487:346374](+)
[346568:346684](+)
[2580484:2580600](-)
[2580781:2583668](-)
[2583918:2585457](-)
[3418645:3418761](-)
[3418942:3421829](-)
[3422079:3423618](-)
[3423993:3424109](-)
[3424290:3427177](-)
[3427427:3428966](-)
[[341494, 343033], [343487, 346374], [346568, 346684]]
[[2580484, 2580600], [2580781, 2583668], [2583918, 2585457], [3418645, 3418761], [3418942, 3421829], [3422079, 3423618], [3423993, 3424109], [3424290, 3427177], [3427427, 3428966]]


Воспользуемся приложенной к ДЗ ссылкой и скачаем .fasta файл (https://www.ncbi.nlm.nih.gov/nuccore/HF680312.1?report=fasta) с полным геномом для бактерии. Сформируем последовательность генома:

In [ ]:
genome = ""

genome_file = open("sequence.fasta", 'r')
genome_file.readline()
for line in genome_file:
  genome += line.replace('\n', '')
genome_file.close()

In [ ]:
rna_file = open("rna.fasta", 'w')
# Сначала работаем с + стрендом, вырезка происходит обычным образом.
for rna in plus:
  # Строка с описанием.
  start = rna[0]
  end = rna[1]
  rna_file.write(f'> rRNA {start}:{end}\n')
  # Непосредственно вырезка.
  rna_file.write(genome[start:end] + '\n')
# Теперь вырезка для - стренда с учетом комплиментарности.
seq = Seq(genome)
seq = seq.reverse_complement()
for rna in minus:
  start = rna[0]
  end = rna[1]
  rna_file.write(f'> rRNA {start}:{end}\n')
  rna_file.write(genome[start:end] + '\n')
rna_file.close()

Используем команду BLASTn:

In [ ]:
!wget https://raw.githubusercontent.com/DaryaSushkova/hse22_hw1/main/data/scaffolds.fasta

--2022-10-19 18:49:33--  https://raw.githubusercontent.com/DaryaSushkova/hse22_hw1/main/data/scaffolds.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3977075 (3.8M) [text/plain]
Saving to: ‘scaffolds.fasta’

scaffolds.fasta     100%[===================>]   3.79M  --.-KB/s    in 0.07s   

2022-10-19 18:49:33 (51.9 MB/s) - ‘scaffolds.fasta’ saved [3977075/3977075]



In [ ]:
!blastn -query rna.fasta -subject scaffolds.fasta > blast_GENOME.gbk

Напишем код для анализа процента сходства:

In [ ]:
curr_per = list()
# Результирующйи словарь выравниваний и процентов.
result = dict(record = list(), percent = list())
# Флаг вывода информации о процентах.
printed = True

blast_file = open("blast_GENOME.gbk", 'r')
for line in blast_file:
  # Извлекаем значения в строке записи.
  values = line.split()

  if not values:
    continue

  # Запись текущего процента.
  if (values[0] == "Identities"):
    curr_per.append(values[3][1:-2:])

  # Строка аннотации скаффолда.
  if (values[0] == '>'):
    if result['record']:
      result['percent'].append(curr_per)
    result['record'].append(values[1])
    curr_per = list()
  
  # Строка аннотации RNA (именно "Query=", просто "Query" содержит последовательности).
  if (values[0] == "Query="):
    result['percent'].append(curr_per)
    if printed:
      rna = values[2]
      printed = False
    if result['record']:
      print(f'rRNA {rna}:')
      # Вывод процентов для всех имеющихся на данный момент записей.
      for index in range(len(result['record'])):
        print(f'{result["record"][index]}: {result["percent"][index]}')
      print("---")
      printed = True
    result = {'record': [], 'percent': []}

rRNA 341494:343033:
scaffold1_cov232: ['100%', '99%', '99%']
scaffold31_cov590: ['100%']
---
rRNA 346568:346684:
scaffold1_cov232: ['100%', '99%', '99%']
scaffold34_cov590: ['100%']
scaffold40_cov590: ['100%']
scaffold48_cov632: ['100%']
---
rRNA 2580484:2580600:
scaffold1_cov232: ['100%', '100%', '100%']
scaffold34_cov590: ['100%']
---
rRNA 2580781:2583668:
scaffold1_cov232: ['98%', '98%', '98%']
scaffold34_cov590: ['99%']
---
rRNA 2583918:2585457:
scaffold1_cov232: ['100%', '99%', '99%']
scaffold34_cov590: ['100%']
scaffold40_cov590: ['100%']
scaffold48_cov632: ['100%']
---
rRNA 3418645:3418761:
scaffold1_cov232: ['99%', '99%', '99%']
scaffold31_cov590: ['99%']
---
rRNA 3418942:3421829:
scaffold1_cov232: ['100%', '100%', '100%']
scaffold34_cov590: ['100%']
---
rRNA 3422079:3423618:
scaffold1_cov232: ['100%', '99%', '99%']
scaffold34_cov590: ['100%']
scaffold40_cov590: ['100%']
scaffold48_cov632: ['100%']
---
rRNA 3423993:3424109:
scaffold1_cov232: ['99%', '99%', '99%']
scaffold31_cov